In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import keras 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras_multi_head import MultiHead
from keras_self_attention import SeqSelfAttention
from sklearn.model_selection import train_test_split
import pandas as pd
import json 
import csv 
import os 
from IPython.display import clear_output
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

Using TensorFlow backend.


In [2]:
weights_file_name = "intent_model.h5" 
epoch_number1 = 100

In [3]:
def json_to_csv_file():
    # Opening JSON file and loading the data 
    # into the variable data 
    with open('intents.json') as json_file: 
        data = json.load(json_file) 
  
    intents_data = data['intents'] 
  
    try:
        os.remove("intents_data.csv") 
    except:
        pass 

    # open csv type file for writing 
    intent_data_file = open('intents_data.csv', 'w', newline='') 
  
    # create the csv writer object 
    csv_writer = csv.writer(intent_data_file) 
  
    # Counter variable used for writing headers to the CSV file 
    count = 0
  
    for intent in intents_data: 
        if count == 0: 
            print(intent)
            # Writing headers of CSV file 
            header = intent.keys() 
            print(header)
            csv_writer.writerow(header) 
            count += 1
  
        # Writing data of CSV file 
        csv_writer.writerow(intent.values()) 
        # file will be automatically properly closed in case of an Exception.
        intent_data_file.flush() 

    intent_data_file.close() 

    data = pd.read_csv("intents_data.csv").rename(columns={"id": "target_intent_or_id", "query": "feature_sentence_or_query"})
    data = data[["feature_sentence_or_query","target_intent_or_id"]]
    data.to_csv("intents_data.csv", index=False)
    data = pd.read_csv("intents_data.csv")
    print("process 1 . ")
    print(data)
  
    data_store = open("intents_data.csv")
    input_file = csv.reader(data_store)

    try:
        del data 
    except:
        pass 

    try:
        os.remove("processed_intents_data.csv") 
    except:
        pass 

    count_check = 0
    
    df = pd.DataFrame(columns=["feature_sentence_or_query", "target_intent_or_id"])
    count_check = 0
    for row in input_file:
        if(count_check == 0):
            count_check = count_check + 1
            pass
        else:
            data_1 = list(row[0].replace("['", "").replace("']", '').replace(" '", "").split('\','))
            print(data_1)
            for data in data_1:
                df = df.append({
            "feature_sentence_or_query": data,
            "target_intent_or_id": row[1]  
             }, ignore_index=True)
        
    df.to_csv("processed_intents_data.csv", index=False)
    data_store.close() 
    os.remove("intents_data.csv") 
    

In [4]:
def load_dataset(filename):
  json_to_csv_file()
  df = pd.read_csv(filename, encoding = "latin1", names = ["feature_sentence_or_query", "target_intent_or_id"],header = 1)
  print(df.head())
  intent = df["target_intent_or_id"]
  unique_intent = list(set(intent))
  sentences = list(df["feature_sentence_or_query"])
  
  return (intent, unique_intent, sentences)
  


In [5]:
def cleaning(sentences):
  words = []
  for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean)
    #stemming
    words.append([i.lower() for i in w])
    
  return words  

In [6]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [7]:
def max_length1(words):
  return(len(max(words, key = len)))
  

In [8]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [9]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [10]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [11]:
def get_feature_and_target_data(intent, unique_intent, cleaned_words, word_tokenizer, vocab_size, max_length):
    
    print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))
    encoded_doc = encoding_doc(word_tokenizer, cleaned_words)
    padded_doc = padding_doc(encoded_doc, max_length)
    #tokenizer with filter changed
    output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')
    print(output_tokenizer.word_index)
    encoded_output = encoding_doc(output_tokenizer, intent)
    encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)
    output_one_hot = one_hot(encoded_output)
    feature = padded_doc
    target = output_one_hot
    return feature, target


In [12]:
def data_insight():
    intent, unique_intent, sentences = load_dataset("processed_intents_data.csv")
    nltk.download("stopwords")
    nltk.download("punkt")
    #define stemmer
    stemmer = LancasterStemmer()
    cleaned_words = cleaning(sentences)
    word_tokenizer = create_tokenizer(cleaned_words)
    vocab_size = len(word_tokenizer.word_index) + 1
    max_length = max_length1(cleaned_words)
    feature, target = get_feature_and_target_data(intent, unique_intent, cleaned_words, word_tokenizer, vocab_size, max_length)
    clear_output()
    X_train, X_test, y_train, y_test = train_test_split(feature, target, shuffle = True, test_size = 0.1)
    
    print("Shape of X_train = %s and y_train = %s" % (X_train.shape, y_train.shape))
    print("Shape of X_test = %s and y_test = %s" % (X_test.shape, y_test.shape))
    return intent, unique_intent, cleaned_words, word_tokenizer, vocab_size, max_length, X_train, X_test, y_train, y_test 


In [13]:
def create_model(vocab_size, max_length, y_train):
  model = Sequential()
  model.add(Embedding(vocab_size, 512, input_length = max_length, trainable = False))
  model.add(MultiHead(LSTM(258), layer_num=6, name='Multi-LSTMs'))  
  #model.add(Dropout(0.4))
  # model.add(BatchNormalization())
  #model.add(MultiHead([  
  #  keras.layers.Conv1D(filters=512, kernel_size=1, padding='same'),
  #  keras.layers.Conv1D(filters=512, kernel_size=1, padding='same'),
  #  keras.layers.Conv1D(filters=512, kernel_size=1, padding='same'),
  #    ], name='Multi-CNNs'))
  #model.add(MultiHead(LSTM(512), layer_num=1, name='Multi-LSTMs'))  
  # model.add(MultiHead(LSTM(128), layer_num=5, name='Multi-LSTMs'))
  model.add(SeqSelfAttention(units=128, attention_width=16, attention_activation='sigmoid'))
  # model.add(BatchNormalization())
  model.add(Bidirectional(LSTM(128, return_sequences=True
                       )))
  #model.add(Dropout(0.001))
  #model.add(BatchNormalization())
  
  model.add(keras.layers.Flatten(name='Flatten'))
  model.add(Dense(128))  
  #model.add(BatchNormalization())
  model.add(Dense(32, activation = "relu"))
  model.add(Dense(y_train.shape[1], activation = "softmax"))
  
  return model

In [14]:
def train_intent_classifier_model(name, weights_file_name, epoch_number, vocab_size, max_length, X_train, X_test, y_train, y_test):
    
    
    print("\n\n\n", name, " started . \n\n\n")
    
    m = create_model(vocab_size, max_length, y_train)

    m.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    clear_output()
    print(m.summary())

    
    checkpoint = ModelCheckpoint(weights_file_name, monitor='val_accuracy', verbose=1, save_weights_only=True, save_best_only=True, mode='max')

    hist = m.fit(X_train, y_train, epochs = epoch_number, batch_size = 2, validation_data = (X_test, y_test), callbacks = [checkpoint])
    
    m = create_model(vocab_size, max_length, y_train)
    m.load_weights(weights_file_name)
    m.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    validation_loss, validation_accuracy  = m.evaluate(X_test, y_test)
    
    print("\n\n-------------------   Model Evaluation . ----------------------\n")
    print(" validation accuracy = ", validation_accuracy, " , validation loss = ", validation_loss, "\n")
    print(90*"-")
    return hist, m

In [15]:
intent, unique_intent, cleaned_words, word_tokenizer, vocab_size, max_length, \
X_train, X_test, y_train, y_test = data_insight()

Shape of X_train = (22, 5) and y_train = (22, 8)
Shape of X_test = (3, 5) and y_test = (3, 8)


In [16]:
hist, m = train_intent_classifier_model("multihead model", weights_file_name, epoch_number1, vocab_size, max_length, X_train, X_test, y_train, y_test)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 512)            19968     
_________________________________________________________________
Multi-LSTMs (MultiHead)      (None, 258, 6)            4774032   
_________________________________________________________________
seq_self_attention_1 (SeqSel (None, 258, 6)            1793      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 258, 256)          138240    
_________________________________________________________________
Flatten (Flatten)            (None, 66048)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8454272   
_________________________________________________________________
dense_2 (Dense)              (None, 32)               


Epoch 00033: val_accuracy did not improve from 1.00000
Epoch 34/100
22/22 [==============================] - 13s 571ms/step - loss: 0.2183 - accuracy: 0.8182 - val_loss: 4.6073 - val_accuracy: 0.3333

Epoch 00034: val_accuracy did not improve from 1.00000
Epoch 35/100
22/22 [==============================] - 13s 574ms/step - loss: 0.2520 - accuracy: 0.8182 - val_loss: 4.4655 - val_accuracy: 0.3333

Epoch 00035: val_accuracy did not improve from 1.00000
Epoch 36/100
22/22 [==============================] - 13s 571ms/step - loss: 0.2245 - accuracy: 0.8182 - val_loss: 4.6107 - val_accuracy: 0.3333

Epoch 00036: val_accuracy did not improve from 1.00000
Epoch 37/100
22/22 [==============================] - 13s 572ms/step - loss: 0.2422 - accuracy: 0.8636 - val_loss: 4.1971 - val_accuracy: 0.3333

Epoch 00037: val_accuracy did not improve from 1.00000
Epoch 38/100
22/22 [==============================] - 13s 573ms/step - loss: 0.2010 - accuracy: 0.8636 - val_loss: 3.6902 - val_accuracy: 0.


Epoch 00074: val_accuracy did not improve from 1.00000
Epoch 75/100
22/22 [==============================] - 13s 570ms/step - loss: 0.2173 - accuracy: 0.8182 - val_loss: 4.2453 - val_accuracy: 0.6667

Epoch 00075: val_accuracy did not improve from 1.00000
Epoch 76/100
22/22 [==============================] - 13s 574ms/step - loss: 0.2037 - accuracy: 0.7727 - val_loss: 4.0533 - val_accuracy: 0.6667

Epoch 00076: val_accuracy did not improve from 1.00000
Epoch 77/100
22/22 [==============================] - 13s 571ms/step - loss: 0.1974 - accuracy: 0.7727 - val_loss: 4.1663 - val_accuracy: 0.3333

Epoch 00077: val_accuracy did not improve from 1.00000
Epoch 78/100
22/22 [==============================] - 13s 572ms/step - loss: 0.2119 - accuracy: 0.7727 - val_loss: 4.2704 - val_accuracy: 0.3333

Epoch 00078: val_accuracy did not improve from 1.00000
Epoch 79/100
22/22 [==============================] - 13s 571ms/step - loss: 0.2061 - accuracy: 0.8182 - val_loss: 4.4605 - val_accuracy: 0.

In [17]:

def predictions(text, m, classes):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = m.predict_proba(x)
  
  
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  print(ids)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
  clear_output()
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



In [18]:
m = create_model(vocab_size, max_length, y_train)
m.load_weights(weights_file_name)
text = "text's intent prediction"
predictions(text, m, unique_intent)

_test3 has confidence = 0.99495924
_test1 has confidence = 0.0039426223
_test4 has confidence = 0.0010607963
_test2 has confidence = 3.0151312e-05
_test5 has confidence = 6.629872e-06
_test9 has confidence = 5.075522e-07
_test7 has confidence = 7.535429e-08
_test6 has confidence = 1.94949e-10


In [19]:
m.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
m.evaluate(X_test, y_test)

3/3 [==============================] - 1s 473ms/step


[0.20717214047908783, 1.0]